# Titanic Survival Predictions Using Neural Netwrok

## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
import random

## Reading the Dataset using Pandas 

In [2]:
df = pd.read_csv("titanic.csv")

In [3]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Dropping Unwanted Columns

In [4]:
df.drop(["Name","Ticket"], axis=1, inplace=True)

## Dealing with Nan Values

In [5]:
df.isnull().mean()

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

In [6]:
df["Sex"] = np.where(df["Sex"]=="male",1,0)

In [7]:
def impute_nan(df,variable):
    df[variable+"_random"] = df[variable]
    random_values = df[variable].dropna().sample(df[variable].isnull().sum(),random_state=0)
    random_values.index = df[df[variable].isnull()].index
    df.loc[df[variable].isnull(),variable+"_random"] = random_values

In [8]:
impute_nan(df,"Age")

In [9]:
df_dummy = pd.get_dummies(df["Embarked"], drop_first=True)

In [10]:
df = pd.concat([df,df_dummy], axis=1)

In [11]:
df.drop(["Age","Embarked"], axis=1,inplace=True)

## Dealing with Categorical Features

In [12]:
df["Cabin"].fillna("Missing", inplace=True)

In [13]:
df["Cabin"] = df["Cabin"].astype(str).str[0]

In [14]:
df["Cabin"].value_counts()

M    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Cabin, dtype: int64

In [15]:
df.groupby("Cabin")["Survived"].mean().sort_values()

Cabin
T    0.000000
M    0.299854
A    0.466667
G    0.500000
C    0.593220
F    0.615385
B    0.744681
E    0.750000
D    0.757576
Name: Survived, dtype: float64

In [16]:
values = df.groupby("Cabin")["Survived"].mean().sort_values().index

In [17]:
values_dict = {k:i for i,k in enumerate(values,0)}

In [18]:
df["Cabin"] = df["Cabin"].map(values_dict)

In [19]:
df.head(10)

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Cabin,Age_random,Q,S
0,1,0,3,1,1,0,7.2500,1,22.0,0,1
1,2,1,1,0,1,0,71.2833,4,38.0,0,0
2,3,1,3,0,0,0,7.9250,1,26.0,0,1
3,4,1,1,0,1,0,53.1000,4,35.0,0,1
4,5,0,3,1,0,0,8.0500,1,35.0,0,1
5,6,0,3,1,0,0,8.4583,1,28.0,1,0
6,7,0,1,1,0,0,51.8625,7,54.0,0,1
7,8,0,3,1,3,1,21.0750,1,2.0,0,1
8,9,1,3,0,0,2,11.1333,1,27.0,0,1
9,10,1,2,0,1,0,30.0708,1,14.0,0,0


## Splitting the Data For Training and Testing 

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X = df.drop(["Survived"], axis=1).values
y = df["Survived"].values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [23]:
from keras.utils import normalize

In [24]:
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [25]:
X_train

array([[0.99934636, 0.00272301, 0.00136151, ..., 0.03131467, 0.        ,
        0.00136151],
       [0.9977271 , 0.00116285, 0.00116285, ..., 0.05930546, 0.        ,
        0.00116285],
       [0.93647037, 0.03426111, 0.01142037, ..., 0.33119074, 0.        ,
        0.01142037],
       ...,
       [0.92312091, 0.00273113, 0.        , ..., 0.11197621, 0.        ,
        0.        ],
       [0.9904233 , 0.00189012, 0.        , ..., 0.08316532, 0.        ,
        0.        ],
       [0.99630906, 0.00345941, 0.        , ..., 0.02882839, 0.        ,
        0.00115314]])

## Creating the Sequential Model Architecture

In [26]:
model = Sequential()
model.add(Dense(200, input_dim=10, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(200, input_dim=200, activation=tf.keras.activations.tanh))
model.add(Dropout(0.2))
model.add(Dense(200, input_dim=200, activation="relu"))
model.add(Dense(1, activation=tf.keras.activations.sigmoid))

In [27]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,loss="binary_crossentropy", metrics=["mse","accuracy"])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2200      
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 82,801
Trainable params: 82,801
Non-trainable params: 0
____________________________________________________

## Fitting the Data into the Model

In [29]:
history = model.fit(X_train,y_train,epochs=2000, batch_size=64, verbose=1, validation_split=0.15)

Epoch 1/2000
10/10 [==============================] - 0s 28ms/step - loss: 0.6671 - mse: 0.2371 - accuracy: 0.6182 - val_loss: 0.6131 - val_mse: 0.2111 - val_accuracy: 0.6916
Epoch 2/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.6568 - mse: 0.2321 - accuracy: 0.6182 - val_loss: 0.6245 - val_mse: 0.2162 - val_accuracy: 0.7103
Epoch 3/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.6557 - mse: 0.2317 - accuracy: 0.6215 - val_loss: 0.6090 - val_mse: 0.2091 - val_accuracy: 0.7103
Epoch 4/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.6519 - mse: 0.2298 - accuracy: 0.6215 - val_loss: 0.6097 - val_mse: 0.2093 - val_accuracy: 0.7009
Epoch 5/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.6558 - mse: 0.2317 - accuracy: 0.6264 - val_loss: 0.6132 - val_mse: 0.2109 - val_accuracy: 0.7103
Epoch 6/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.6485 - mse: 0.2282 - accuracy: 0.6380 - val_loss: 0.5

Epoch 48/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5363 - mse: 0.1785 - accuracy: 0.7355 - val_loss: 0.4657 - val_mse: 0.1496 - val_accuracy: 0.7850
Epoch 49/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5321 - mse: 0.1779 - accuracy: 0.7339 - val_loss: 0.5231 - val_mse: 0.1699 - val_accuracy: 0.7570
Epoch 50/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5521 - mse: 0.1854 - accuracy: 0.7289 - val_loss: 0.5909 - val_mse: 0.2001 - val_accuracy: 0.7664
Epoch 51/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5843 - mse: 0.1998 - accuracy: 0.6975 - val_loss: 0.4682 - val_mse: 0.1502 - val_accuracy: 0.7850
Epoch 52/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5365 - mse: 0.1787 - accuracy: 0.7322 - val_loss: 0.4751 - val_mse: 0.1523 - val_accuracy: 0.7850
Epoch 53/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5345 - mse: 0.1786 - accuracy: 0.7289 - val_loss

Epoch 95/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4965 - mse: 0.1628 - accuracy: 0.7537 - val_loss: 0.4612 - val_mse: 0.1485 - val_accuracy: 0.7757
Epoch 96/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4848 - mse: 0.1590 - accuracy: 0.7669 - val_loss: 0.4606 - val_mse: 0.1490 - val_accuracy: 0.7850
Epoch 97/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4948 - mse: 0.1621 - accuracy: 0.7587 - val_loss: 0.4404 - val_mse: 0.1413 - val_accuracy: 0.8037
Epoch 98/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4838 - mse: 0.1589 - accuracy: 0.7504 - val_loss: 0.4278 - val_mse: 0.1372 - val_accuracy: 0.7944
Epoch 99/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5001 - mse: 0.1648 - accuracy: 0.7521 - val_loss: 0.4466 - val_mse: 0.1437 - val_accuracy: 0.7944
Epoch 100/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.5015 - mse: 0.1646 - accuracy: 0.7537 - val_los

10/10 [==============================] - 0s 8ms/step - loss: 0.4603 - mse: 0.1497 - accuracy: 0.7702 - val_loss: 0.4406 - val_mse: 0.1421 - val_accuracy: 0.7850
Epoch 142/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.4775 - mse: 0.1552 - accuracy: 0.7769 - val_loss: 0.4334 - val_mse: 0.1396 - val_accuracy: 0.8037
Epoch 143/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4490 - mse: 0.1453 - accuracy: 0.7752 - val_loss: 0.4321 - val_mse: 0.1389 - val_accuracy: 0.8131
Epoch 144/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4742 - mse: 0.1556 - accuracy: 0.7620 - val_loss: 0.4501 - val_mse: 0.1446 - val_accuracy: 0.7850
Epoch 145/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4645 - mse: 0.1516 - accuracy: 0.7636 - val_loss: 0.4437 - val_mse: 0.1438 - val_accuracy: 0.7757
Epoch 146/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4582 - mse: 0.1475 - accuracy: 0.7818 - val_loss: 0.4793 

Epoch 188/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4458 - mse: 0.1442 - accuracy: 0.7868 - val_loss: 0.4160 - val_mse: 0.1344 - val_accuracy: 0.8131
Epoch 189/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4506 - mse: 0.1456 - accuracy: 0.8033 - val_loss: 0.4151 - val_mse: 0.1333 - val_accuracy: 0.8224
Epoch 190/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4396 - mse: 0.1416 - accuracy: 0.8017 - val_loss: 0.4289 - val_mse: 0.1393 - val_accuracy: 0.8131
Epoch 191/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4409 - mse: 0.1414 - accuracy: 0.7934 - val_loss: 0.4359 - val_mse: 0.1401 - val_accuracy: 0.7944
Epoch 192/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.4442 - mse: 0.1435 - accuracy: 0.7868 - val_loss: 0.4711 - val_mse: 0.1532 - val_accuracy: 0.7664
Epoch 193/2000
10/10 [==============================] - 0s 8ms/step - loss: 0.4747 - mse: 0.1549 - accuracy: 0.7967 - va

...

10/10 [==============================] - 0s 6ms/step - loss: 0.3080 - mse: 0.0965 - accuracy: 0.8645 - val_loss: 1.1122 - val_mse: 0.1510 - val_accuracy: 0.7944
Epoch 1801/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.2984 - mse: 0.0937 - accuracy: 0.8661 - val_loss: 1.2062 - val_mse: 0.1539 - val_accuracy: 0.8131
Epoch 1802/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3027 - mse: 0.0966 - accuracy: 0.8744 - val_loss: 1.1208 - val_mse: 0.1476 - val_accuracy: 0.8318
Epoch 1803/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.2961 - mse: 0.0920 - accuracy: 0.8810 - val_loss: 1.1228 - val_mse: 0.1594 - val_accuracy: 0.7944
Epoch 1804/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.2954 - mse: 0.0902 - accuracy: 0.8694 - val_loss: 1.1209 - val_mse: 0.1453 - val_accuracy: 0.8318
Epoch 1805/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.3119 - mse: 0.0976 - accuracy: 0.8661 - val_loss: 1.

10/10 [==============================] - 0s 5ms/step - loss: 0.2966 - mse: 0.0933 - accuracy: 0.8628 - val_loss: 1.2909 - val_mse: 0.1468 - val_accuracy: 0.8318
Epoch 1847/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.3030 - mse: 0.0909 - accuracy: 0.8711 - val_loss: 1.3121 - val_mse: 0.1494 - val_accuracy: 0.8318
Epoch 1848/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3097 - mse: 0.0986 - accuracy: 0.8628 - val_loss: 1.3250 - val_mse: 0.1508 - val_accuracy: 0.8037
Epoch 1849/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3205 - mse: 0.0940 - accuracy: 0.8711 - val_loss: 1.3114 - val_mse: 0.1534 - val_accuracy: 0.8131
Epoch 1850/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3119 - mse: 0.0984 - accuracy: 0.8694 - val_loss: 1.4255 - val_mse: 0.1516 - val_accuracy: 0.8318
Epoch 1851/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3142 - mse: 0.0986 - accuracy: 0.8579 - val_loss: 1.

10/10 [==============================] - 0s 5ms/step - loss: 0.2986 - mse: 0.0939 - accuracy: 0.8694 - val_loss: 1.2996 - val_mse: 0.1394 - val_accuracy: 0.8505
Epoch 1893/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.2919 - mse: 0.0909 - accuracy: 0.8793 - val_loss: 1.3068 - val_mse: 0.1519 - val_accuracy: 0.8037
Epoch 1894/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.2755 - mse: 0.0856 - accuracy: 0.8826 - val_loss: 1.3096 - val_mse: 0.1565 - val_accuracy: 0.7850
Epoch 1895/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.3042 - mse: 0.0947 - accuracy: 0.8628 - val_loss: 1.2572 - val_mse: 0.1494 - val_accuracy: 0.8037
Epoch 1896/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.2893 - mse: 0.0903 - accuracy: 0.8744 - val_loss: 1.3462 - val_mse: 0.1509 - val_accuracy: 0.8224
Epoch 1897/2000
10/10 [==============================] - 0s 5ms/step - loss: 0.2838 - mse: 0.0877 - accuracy: 0.8893 - val_loss: 1.

10/10 [==============================] - 0s 8ms/step - loss: 0.3118 - mse: 0.0961 - accuracy: 0.8744 - val_loss: 1.2621 - val_mse: 0.1356 - val_accuracy: 0.8411
Epoch 1939/2000
10/10 [==============================] - 0s 10ms/step - loss: 0.2966 - mse: 0.0929 - accuracy: 0.8661 - val_loss: 1.2600 - val_mse: 0.1409 - val_accuracy: 0.8131
Epoch 1940/2000
10/10 [==============================] - 0s 7ms/step - loss: 0.2983 - mse: 0.0931 - accuracy: 0.8760 - val_loss: 1.2865 - val_mse: 0.1409 - val_accuracy: 0.8411
Epoch 1941/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.2949 - mse: 0.0944 - accuracy: 0.8678 - val_loss: 1.3525 - val_mse: 0.1570 - val_accuracy: 0.8224
Epoch 1942/2000
10/10 [==============================] - 0s 8ms/step - loss: 0.2983 - mse: 0.0930 - accuracy: 0.8793 - val_loss: 1.2739 - val_mse: 0.1460 - val_accuracy: 0.8411
Epoch 1943/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.3069 - mse: 0.0970 - accuracy: 0.8645 - val_loss: 1

10/10 [==============================] - 0s 6ms/step - loss: 0.2948 - mse: 0.0907 - accuracy: 0.8678 - val_loss: 1.2565 - val_mse: 0.1492 - val_accuracy: 0.7944
Epoch 1985/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.2949 - mse: 0.0939 - accuracy: 0.8661 - val_loss: 1.2735 - val_mse: 0.1463 - val_accuracy: 0.8318
Epoch 1986/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.3044 - mse: 0.0981 - accuracy: 0.8612 - val_loss: 1.2604 - val_mse: 0.1471 - val_accuracy: 0.8411
Epoch 1987/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.2918 - mse: 0.0902 - accuracy: 0.8760 - val_loss: 1.2518 - val_mse: 0.1568 - val_accuracy: 0.7757
Epoch 1988/2000
10/10 [==============================] - 0s 6ms/step - loss: 0.3124 - mse: 0.0962 - accuracy: 0.8744 - val_loss: 1.2186 - val_mse: 0.1464 - val_accuracy: 0.8411
Epoch 1989/2000
10/10 [==============================] - 0s 7ms/step - loss: 0.2827 - mse: 0.0873 - accuracy: 0.8793 - val_loss: 1.

## Predictions

In [30]:
def predict(row):
    num = df.drop(["Survived"], axis=1).iloc[row].values.reshape(1,-1)
    prediction = model.predict(num)[0][0]
    actual = df.iloc[row]["Survived"]
    print("\n")
    print("The predicted value of row {} is {} and the actual value is {}".format(row,prediction,actual))

In [31]:
for _ in range(10):
    row = random.randint(0,890)
    predict(row)



The predicted value of row 170 is 0.0 and the actual value is 0.0


The predicted value of row 591 is 1.0 and the actual value is 1.0


The predicted value of row 625 is 4.503384133587122e-23 and the actual value is 0.0


The predicted value of row 525 is 0.0 and the actual value is 0.0


The predicted value of row 250 is 0.0 and the actual value is 0.0


The predicted value of row 469 is 1.0 and the actual value is 1.0


The predicted value of row 218 is 0.0 and the actual value is 1.0


The predicted value of row 46 is 0.0 and the actual value is 0.0


The predicted value of row 192 is 0.0 and the actual value is 1.0


The predicted value of row 452 is 0.0 and the actual value is 0.0


## Saving the Model

In [32]:
model.save("Titanic")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Titanic\assets
